In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
bracket_df = pd.read_csv('../Data_cleaning/barttorvik_all_years.csv')
bracket_df.head()

,Year,Seed,School,Round Finished,Conf,G,Rec,AdjOE,AdjDE,AdjNR,...,FTR,FTRD,2P%,2P%D,3P%,3P%D,3PR,3PRD,Adj T.,WAB
0,2008,1,Kansas,6.0,B12,33,30–3,121.1,85.9,35.2,...,37.5,30.8,54.8,40.9,39.9,34.0,29.2,38.1,69.5,9.9
1,2008,1,Memphis,5.0,CUSA,34,33–1,113.4,84.0,29.4,...,38.6,32.7,53.3,41.6,35.0,29.8,36.7,28.7,70.7,9.0
2,2008,1,UCLA,4.0,P10,33,30–3,116.4,87.2,29.2,...,38.0,25.7,52.9,46.8,33.8,34.2,28.1,27.9,66.2,10.7
3,2008,3,Wisconsin,2.0,B10,33,29–4,112.3,85.2,27.1,...,39.3,25.4,49.2,41.3,35.9,31.1,31.1,32.8,63.5,8.2
4,2008,2,Duke,1.0,ACC,32,27–5,117.3,89.0,28.3,...,40.4,32.0,51.8,47.1,38.4,32.8,39.1,24.8,73.7,8.9


In [2]:
#Exclude 2025 data from CSV

#Create Copy of Data
copy_df = bracket_df.copy()

#Create New Dataframe for all years except 2025
no_2025_df = copy_df.loc[(copy_df["Year"] != 2025)]


In [3]:
#Use list comprehension to create Sweet 16 values

s_16 = [1 if x >= 2 else 0 for x in no_2025_df['Round Finished']]

#Assign list to column
no_2025_df['Sweet 16'] = s_16
#Show Dataframe
no_2025_df.head()

/var/folders/f6/crwn6xns7ng5zdqghg60td240000gn/T/ipykernel_15667/1305749034.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_2025_df['Sweet 16'] = s_16


,Year,Seed,School,Round Finished,Conf,G,Rec,AdjOE,AdjDE,AdjNR,...,FTRD,2P%,2P%D,3P%,3P%D,3PR,3PRD,Adj T.,WAB,Sweet 16
0,2008,1,Kansas,6.0,B12,33,30–3,121.1,85.9,35.2,...,30.8,54.8,40.9,39.9,34.0,29.2,38.1,69.5,9.9,1
1,2008,1,Memphis,5.0,CUSA,34,33–1,113.4,84.0,29.4,...,32.7,53.3,41.6,35.0,29.8,36.7,28.7,70.7,9.0,1
2,2008,1,UCLA,4.0,P10,33,30–3,116.4,87.2,29.2,...,25.7,52.9,46.8,33.8,34.2,28.1,27.9,66.2,10.7,1
3,2008,3,Wisconsin,2.0,B10,33,29–4,112.3,85.2,27.1,...,25.4,49.2,41.3,35.9,31.1,31.1,32.8,63.5,8.2,1
4,2008,2,Duke,1.0,ACC,32,27–5,117.3,89.0,28.3,...,32.0,51.8,47.1,38.4,32.8,39.1,24.8,73.7,8.9,0


In [4]:
#Drop Round Finished from DataFrame
sweet_16_df = no_2025_df.drop(columns='Round Finished')

sweet_16_df.head()

,Year,Seed,School,Conf,G,Rec,AdjOE,AdjDE,AdjNR,Barthag,...,FTRD,2P%,2P%D,3P%,3P%D,3PR,3PRD,Adj T.,WAB,Sweet 16
0,2008,1,Kansas,B12,33,30–3,121.1,85.9,35.2,0.9810,...,30.8,54.8,40.9,39.9,34.0,29.2,38.1,69.5,9.9,1
1,2008,1,Memphis,CUSA,34,33–1,113.4,84.0,29.4,0.9692,...,32.7,53.3,41.6,35.0,29.8,36.7,28.7,70.7,9.0,1
2,2008,1,UCLA,P10,33,30–3,116.4,87.2,29.2,0.9653,...,25.7,52.9,46.8,33.8,34.2,28.1,27.9,66.2,10.7,1
3,2008,3,Wisconsin,B10,33,29–4,112.3,85.2,27.1,0.9599,...,25.4,49.2,41.3,35.9,31.1,31.1,32.8,63.5,8.2,1
4,2008,2,Duke,ACC,32,27–5,117.3,89.0,28.3,0.9597,...,32.0,51.8,47.1,38.4,32.8,39.1,24.8,73.7,8.9,0


In [5]:
#Split Record into Wins and Losses
sweet_16_df[['Wins', 'Losses']] = sweet_16_df['Rec'].str.split('–', expand=True)

#Change Columns to Ints
sweet_16_df['Wins'] = sweet_16_df['Wins'].astype(int)
sweet_16_df['Losses'] = sweet_16_df['Losses'].astype(int)

#Drop Rec Column
final_df = sweet_16_df.drop(columns=['Rec', 'School', 'Conf'])

final_df.head()

,Year,Seed,G,AdjOE,AdjDE,AdjNR,Barthag,EFG%,EFGD%,TOR,...,2P%D,3P%,3P%D,3PR,3PRD,Adj T.,WAB,Sweet 16,Wins,Losses
0,2008,1,33,121.1,85.9,35.2,0.9810,56.3,44.8,18.7,...,40.9,39.9,34.0,29.2,38.1,69.5,9.9,1,30,3
1,2008,1,34,113.4,84.0,29.4,0.9692,53.0,42.5,17.2,...,41.6,35.0,29.8,36.7,28.7,70.7,9.0,1,33,1
2,2008,1,33,116.4,87.2,29.2,0.9653,52.3,48.0,18.8,...,46.8,33.8,34.2,28.1,27.9,66.2,10.7,1,30,3
3,2008,3,33,112.3,85.2,27.1,0.9599,50.6,43.1,19.4,...,41.3,35.9,31.1,31.1,32.8,63.5,8.2,1,29,4
4,2008,2,32,117.3,89.0,28.3,0.9597,54.1,47.6,18.2,...,47.1,38.4,32.8,39.1,24.8,73.7,8.9,0,27,5


In [6]:
#Create Dummies Dataframe
dummies_df = pd.get_dummies(final_df)
dummies_df.head()

,Year,Seed,G,AdjOE,AdjDE,AdjNR,Barthag,EFG%,EFGD%,TOR,...,2P%D,3P%,3P%D,3PR,3PRD,Adj T.,WAB,Sweet 16,Wins,Losses
0,2008,1,33,121.1,85.9,35.2,0.9810,56.3,44.8,18.7,...,40.9,39.9,34.0,29.2,38.1,69.5,9.9,1,30,3
1,2008,1,34,113.4,84.0,29.4,0.9692,53.0,42.5,17.2,...,41.6,35.0,29.8,36.7,28.7,70.7,9.0,1,33,1
2,2008,1,33,116.4,87.2,29.2,0.9653,52.3,48.0,18.8,...,46.8,33.8,34.2,28.1,27.9,66.2,10.7,1,30,3
3,2008,3,33,112.3,85.2,27.1,0.9599,50.6,43.1,19.4,...,41.3,35.9,31.1,31.1,32.8,63.5,8.2,1,29,4
4,2008,2,32,117.3,89.0,28.3,0.9597,54.1,47.6,18.2,...,47.1,38.4,32.8,39.1,24.8,73.7,8.9,0,27,5


In [7]:
#separate Data into features and target arrays
y = dummies_df["Sweet 16"]
X = dummies_df.drop(columns="Sweet 16")

#Split the data into training and testing dataset\
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [8]:
#scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
# Create Logistic Regression Model
lr_model = LogisticRegression(solver='lbfgs',
                                max_iter=50,
                                class_weight ='balanced',
                                random_state=1)
lr_model

LogisticRegression(class_weight='balanced', max_iter=50, random_state=1)

In [10]:
lr_model.fit(X_train_scaled, y_train)

LogisticRegression(class_weight='balanced', max_iter=50, random_state=1)

In [11]:
#Score Model  
print(f"Training Data Score: {lr_model.score(X_train, y_train)}")
print(f"Testing Data Score: {lr_model.score(X_test, y_test)}")

Training Data Score: 0.7639060568603214
Testing Data Score: 0.7592592592592593


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [12]:
#Make Predictions
predictions = lr_model.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
results.value_counts()

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


Prediction  Actual
0           0         205
            1          65
Name: count, dtype: int64

In [13]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm_df

,Predicted 0,Predicted 1
Actual 0,205,0
Actual 1,65,0
